In [2]:
# Импортирование библиотек для иницализации основных параметров системы
from fortran_calc_tools import *

# Импортирование неодходимых библиотек
from os import path 
import time
import pandas as pd, numpy as np
from collections import namedtuple
from calc_tools import pairwise_1, pairwise_2
from scipy.linalg import logm, expm, sqrtm, eig, lu, lu_factor, lu_solve
from multiprocessing import Pool
from numpy.linalg import inv
from numpy import linalg as LA

# Импортирование библиотек для визуализации данных !!!!!
import matplotlib.pyplot as plot
import matplotlib as mpl
import seaborn as sns
import plotly as pt
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

In [3]:
# Задание значении характеристик материала для фаз и троса
#                         GM    OPM    S     XA   YA
Faza_A = Fazy("Фаза А", 35.336, 1.0, 150.0, 0.0, 19.0)
Faza_B = Fazy("Фаза В", 35.336, 1.0, 150.0, 6.3, 19.0)
Faza_C = Fazy("Фаза С", 35.336, 1.0, 150.0, 4.2, 25.0)
Tross = Fazy("Тросс", 17.336, 4000.0, 50.0, 2.1, 28.0)

# В данном случае у нас будет 50 гармоник от [1 до 50]
garmoniki = Garmoniki(50)

# Создание объекта для проведения расчетов
rashet = Rachety()

# Инициализация объекта для подстанции "Зеленая" и чтение файла
basepath = path.abspath('')
filepath = path.abspath(path.join(basepath, "..", path.join("data", "Данные ПС Сарепта1.xls")))
PodS_zelenaya = PodStans(filepath, volt=110, harm_num=49)

dummy_str_input = "fuzzBuzz for testing"
dummy_int_input = 404

system = System(PodS_zelenaya, garmoniki, rashet, Fazy, dummy_str_input, dummy_str_input, dummy_int_input, dummy_int_input,
     dummy_int_input, dummy_int_input, dummy_int_input)

# Установка конфигурации для выведения данных
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [4]:
#                   !!! DEBUGGING PROCESS HERE !!!

In [29]:
# GETTING MAIN LINE CHARACTERISTICS FROM EXCEL FILE
UM, FUM, AIM, FIM, funu, fu, funi, fi, knsu, knsi, rmsu, rmsi = system.line_matrix(1)

In [30]:
# DECLARING VARIABLES

WD = np.zeros((2, 50))
PD = np.zeros((3, 700))

UM1 = np.zeros((700, UM.shape[1],UM.shape[2]))
UM2 = np.zeros((700, UM.shape[1],UM.shape[2]))
AIM1 = np.zeros((700, UM.shape[1],UM.shape[2]))
AIM2 = np.zeros((700, UM.shape[1],UM.shape[2]))
PPR1, PPR2 = np.zeros((700)), np.zeros((700))

PPP = np.zeros((1000, 50)); PPP1 = np.zeros((1000, 50));
PPP2 = np.zeros((1000, 50)); PPP3 = np.zeros((1000, 50));
PPP4 = np.zeros((1000, 50)); PPP5 = np.zeros((1000, 50));
PPP6 = np.zeros((1000, 50)); PPP7 = np.zeros((1000, 50));
PPP8 = np.zeros((1000, 50));

# !!!!!!!!!!!!!!!!! CordN = 8 was in original here 
cordN = 4
UK1 = np.array([complex(0, 0) for _ in range(cordN)], dtype="complex128")
AIK1 = np.array([complex(0, 0) for _ in range(cordN)], dtype="complex128")

UK10 = complex(0, 0); UK11 = complex(0, 0); UK12 = complex(0, 0);
AIK10 = complex(0, 0); AIK11 = complex(0, 0); AIK12 = complex(0, 0);
AL = complex(0, 0);

PP1 = 0.0
PP2 = 0.0
RZ = 35.3

In [31]:
# Implementing RASCHET fortran's subroutine in python
def RASCHET(NN, LL):
# LL is in range [0-559] NN [0-49]

#     if NN == 1:
#         raise ValueError
        
#     if LL > 49:
#         LL = 49
#     print(NN, LL)
    global UK1, AIK1, PPP,PP1,PP2,PPP1,PPP2,PPP3,PPP4,PPP5,PPP6,PPP7,PPP8, MM,M,M1,MT,M10,M20,PR,K1,K2,K3,N1,N2,N3,MPR,MTR,MMT
    
    
    XA, YA, OMP, GM, S = Fazy.construc_matrices()
    R0, R, UXM, HI, R11, DET2, DET4 = np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M))
    EVU, B, UX, AIX, SM, EVI, DET1, DET3, BB, AIXM, AA = np.zeros((M)), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M)), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M)), np.zeros((M)), np.zeros((M)),
    B1, B4, B5 = np.zeros((M20), dtype="complex128"), np.zeros((M20)), np.zeros((M20), dtype="complex128")
    B6, B10, B7 = np.zeros((M10)), np.zeros((M10), dtype="complex128"), np.zeros((M10)) 
    
    DET10, DET20, SS, SS1, EX1,  = [], [], [], [], []
    
    HC1, HC2, HC3, HC4, F10, XL, XL1, G, D, HC, UXM = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)) 
    Z, Y, AU, E,  AAI, F, F1 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    F2, D1, D2, D3, LU, LI = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    LU1, LI1, LU2, LU3, LI2, LI3 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    F3, F4, F5, F6, F7 = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M))
    HH13, HH14, HH21, HH22, HH23 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    HH31, HH32, HH33, HH34, HH24 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    HH41, HH42, HH43, HH44, HH = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M10, M10))
    CC, DD, HH11, HH12 = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    A1, A2 = np.zeros((M1, M1), dtype="complex128"), np.zeros((M1, M1), dtype="complex128")
    
    AG, GG = np.zeros((M1, M, M), dtype=type), np.zeros((M20, M20), dtype="complex128")
    GG1, GG2, GG3, GG4, GG5 = np.zeros((M20, M20), dtype="complex128"), np.zeros((M20, M20), dtype="complex128"), np.zeros((M10, M20), dtype="complex128"), np.zeros((M10, M10), dtype="complex128"), np.zeros((M10, M10), dtype="complex128")
    IPVT1, IH = np.zeros((M1)), np.array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
    
    if PR == 1:
        PP1 = 0
    if PR == 2:
        PP2 = 0
#     PPP[NN, LL] = 0
#     PPP1[NN, LL] = 0
#     PPP2[NN, LL] = 0
#     PPP3[NN, LL] = 0
#     PPP4[NN, LL] = 0
#     PPP5[NN, LL] = 0
#     PPP6[NN, LL] = 0
#     PPP7[NN, LL] = 0
#     PPP8[NN, LL] = 0
    
    MMT = MM / MT
    W = float(LL+1)
    EX1 = complex(2.71828, 0)
    
#     print("AIK1: ", AIK1) # LEFT HERE !!!!
    
    # WRITING INTO UNIT 5. LINE MAIN CHARACTERISTIC
    
    for i in range(M):
        R[i]=np.sqrt(S[i]/np.pi)/1000.
        HI[i]=R[i]/(2.)*np.sqrt(2*np.pi*W*50*4*np.pi*OMP[i]*GM[i]/20)
        R0[i]=1000/(GM[i]*S[i])
        if HI[i] < 1:
            R11[i]=R0[i]*(1+HI[i]**4/3)
        if HI[i] > 1: 
            R11[i]=R0[i]*(HI[i]+0.25+3./(64.*HI[i]))
        if i+1 == M:
            pass
            # WRITE R11's values INTO THE FILE "R11"

#     print("W: ", W)
    
    # 845 - 12
    for i in range(M10):
        for j in range(M10):
            HH[i,j] = 0 

    # 12 - 161
    for i in range(M):
        for j in range(M):
            if i == j:
                D[i,i] = R[i]
            if i != j:
                D[i,j]=np.sqrt((XA[i]-XA[j])**2+(YA[i]-YA[j])**2)
            HC[i,j]=np.sqrt((XA[i]-XA[j])**2+(YA[i]+YA[j])**2)
            E[i,j]=complex(0.0, 0.0)
            E[i,i]=complex(1.0, 0.0)

    # 161 - 740
    for i in range(M):
        for j in range(M):
            XL1[i,j]=(0.145*np.log10(1000/D[i,j]))/(100*np.pi)
    
    # 740 - 743
    for i in range(M):
        for j in range(M):
            HC1[i,j]=41.4*10**6*np.log10(HC[i,j]/D[i,j])
    
    # 743 - 744
    HC1 = HC1.astype('float64')
    HC3 = DLINRG(HC1)
    F10 = DMRRRR(HC1,HC3)
    
    for i in range(M):
        for j in range(M):
            HC2[i,j]=HC3[i,j]*2.*np.pi*50
    
    # 744 - 847
    for i in range(M):
        for j in range(M):
            XL[i,j]=XL1[i,j]*W*2*50*np.pi
            HC4[i,j]=HC2[i,j]*W
            R10=0.0
            if i == j:
                Z[i,j]=complex(R11[i],XL[i,j]) 
            if i != j:
                Z[i,j]=complex(R10,XL[i,j])
            if i == j:
                G[i,j]=0.00000004*YA[i]/YA[i] 
            if i != j:
                G[i,j]=-0.00000004*YA[0]/D[i,j] 
            G[i,j]=0.
            Y[i,j]=complex(G[i,j],HC4[i,j])
    
    
    # 847 - the end of subroutine        
    for III in range(MT): # <----- main loop # 1300
#         print("Main loop counter MT: ", III)
        
        if M != 3: # GOTO 767
#             print("Here 3")
            if (M != 4): # GOTO 768
#                 print("Here 4")
                if (M != 6): # GOTO 769
#                     print("Here 6")
                    if (M != 7): # GOTO 770
#                         print("Here 7")
                        if (M != 8): # GOTO 771
                            pass
#                             print("Here 8")

                        else:
                            print("else 8")
                            # 770 - 766
                            for i in range(3):
                                B5[i]=UK1[i]
                                B5[i+3]=UK1[i]
                                B5[MPR+1-1]=complex(0.,0.)
                                B5[MPR+2-1]=complex(0.,0.)
                                B5[i+M]=AIK1[i]
                                B5[i+M+3]=AIK1[i]
                                B5[2*M-1-1]=complex(0.,0.)
                                B5[2*M-1]=complex(0.,0.)
                                B5[i+2*M]=complex(0.,0.)
                                B5[i+2*M+3]=complex(0.,0.)
                                B5[3*M-1-1]=complex(0.,0.)
                                B5[3*M-1]=complex(0.,0.)
                                B5[i+3*M]=complex(0.,0.)
                                B5[i+3*M+3]=complex(0.,0.)
                                B5[4*M-1-1]=complex(0.,0.)
                                B5[4*M-1]=complex(0.,0.)
                          
                    else:
                        print("else 7")
                        # 769 - 765
                        for i in range(3):
                            B5[i]=UK1[i]
                            B5[i+3]=UK1[i] 
                            B5[M-1]=complex(0.,0.) 
                            B5[i+M]=AIK1[i]
                            B5[i+M+3]=AIK1[i] 
                            B5[2*M-1]=complex(0.,0.)
                            B5[i+2*M]=complex(0.,0.)
                            B5[i+2*M+3]=complex(0.,0.)
                            B5[3*M-1]=complex(0.,0.)
                            B5[i+3*M]=complex(0.,0.)
                            B5[i+3*M+3]=complex(0.,0.)
                            B5[4*M-1]=complex(0.,0.)
                      

                else:
                    print("else 6")
                    # 768 - 764
                    for i in range(3):
                        B5[i]=UK1[i]
                        B5[i+3]=UK1[i]
                        B5[i+M]=AIK1[i]
                        B5[i+M+3]=AIK1[i]
                        B5[i+2*M]=complex(0.,0.)
                        B5[i+2*M+3]=complex(0.,0.)
                        B5[i+3*M]=complex(0.,0.)
                        B5[i+3*M+3]=complex(0.,0.)

            else:
#                 print("else 4")
                # 767 - 762
                for i in range(3):
    #   !!!!!!!!!!!!!!!!!!!!   AIK1 and UK1 ARE THE REAL EVILS IN THIS PARTY !!!!!!!!!!!!!!!!!!!!!!!!
#                     print("AIK1[i]: ", AIK1[i])
#                     print("UK1[i]: ",UK1[i])
                    B5[i]=UK1[i]
                    B5[M-1]=complex(0.,0.)
                    B5[i+M]=AIK1[i]
                    B5[2*M-1]=complex(0.,0.)
                    B5[i+2*M]=complex(0.,0.)
                    B5[3*M-1]=complex(0.,0.)
                    B5[i+3*M]=complex(0.,0.)
                    B5[4*M-1]=complex(0.,0.)
                
        else:
            print("else 3")
            # 847 - 761
            for i in range(3):
                B5[i]=UK1[i]
                B5[i+3]=AIK1[i]
                B5[i+6]=complex(0.,0.)
                B5[i+9]=complex(0.,0.)
            
# &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& DEBUGGING &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
         
       #   !!!!!!!!!!!!!!!!!!!!   B5 IS THE REAL EVIL IN THIS PARTY !!!!!!!!!!!!!!!!!!!!!!!!
#         print("B5: ", B5)
        
        AU = DMCRCR(Z,Y)
        SS1=np.sqrt(AU[0,0])
        EVU = DEVLCG(AU)
        
        # IMPORTANT POINT
        for j in range(M):
            for i in range(M):
                F[i,j]=EVU[i]**j # (j-1) was in the first case
                F1[i,j]=F[i,j]
        
        for i in range(M1):
            for j in range(M1):
                A1[i,j]=F[i,j]

        A2 = DLFTCG(A1)
        SS = DET10 = DLFDCG(A2); DET20 = 1 # parameters DET10,DET20 were in function call
        # SS=DET10*(10.**DET20)
        
        for j in range(M1):

            for ii in range(M1):
                for jj in range(M1):
                    F[ii,jj]=F1[ii,jj]

            for i in range(M1):
                F[i,j]=EVU[i]**0.5

            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET1[j] = DLFDCG(A2); DET2[j] = 1 # parameters DET1(J), DET2(J) were in function call

        F2 = DMCRCR(AU,AU)
        F3 = DMCRCR(F2,AU)
        F4 = DMCRCR(F3,AU)
        F5 = DMCRCR(F4,AU)
        F6 = DMCRCR(F5,AU)
        F7 = DMCRCR(F6,AU)
        
        for i in range(M1):
            for ii in range(M):
                for jj in range(M):

                    if i == 0:
                        AG[0,ii,jj]=E[ii,jj]
                    if i == 1:
                        AG[1,ii,jj]=AU[ii,jj]
                    if i == 2:
                        AG[2,ii,jj]=F2[ii,jj]
                    if i == 3:
                        AG[3,ii,jj]=F3[ii,jj]
                    if i == 4:
                        AG[4,ii,jj]=F4[ii,jj]
                    if i == 5:
                        AG[5,ii,jj]=F5[ii,jj]
                    if i == 6:
                        AG[6,ii,jj]=F6[ii,jj]
                    if i == 7:
                        AG[7,ii,jj]=F7[ii,jj]

        for ii in range(M):
            for jj in range(M):
                LU[ii,jj]=0.

        for i in range(M1):
            DET3[i]=DET1[i]/DET10
            DET4[i]=DET2[i]-DET20
            for ii in range(M):
                for jj in range(M):
                    LU[ii,jj]=LU[ii,jj]+AG[i,ii,jj]*DET3[i]*(10**DET4[i])
        
        F3 = DMCRCR(LU,LU)
        AAI = DMCRCR(Y,Z)
        EVI = DEVLCG(AAI)
  
        for j in range(M):
            for i in range(M):
                F[i,j] = EVI[i]**j # j-1
                F1[i,j] = F[i,j]

        for i in range(M1):
            for j in range(M1):
                A1[i,j]=F[i,j]

        A2 = DLFTCG(A1)
        DET10 = DLFDCG(A2); DET20 = 1 # parameters DET10,DET20 were in function call
        
        for j in range(M1):
            for ii in range(M1):
                for jj in range(M1):
                    F[ii,jj]=F1[ii,jj]

            for i in range(M1):
                F[i,j]=EVI[i]**0.5
            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET1[j] = DLFDCG(A2); DET2[j] = 1 # parameters DET1(J),DET2(J) were in function call
            
        F2 = DMCRCR(AAI,AAI)
        F3 = DMCRCR(F2,AAI,)
        F4 = DMCRCR(F3,AAI)
        F5 = DMCRCR(F4,AAI)
        F6 = DMCRCR(F5,AAI)
        F7 = DMCRCR(F6,AAI)

        for i in range(M1):
            for ii in range(M):
                for jj in range(M):

                    if i == 0:
                        AG[0,ii,jj]=E[ii,jj]
                    if i == 1:
                        AG[1,ii,jj]=AAI[ii,jj]
                    if i == 2:
                        AG[2,ii,jj]=F2[ii,jj]
                    if i == 3:
                        AG[3,ii,jj]=F3[ii,jj]
                    if i == 4:
                        AG[4,ii,jj]=F4[ii,jj]
                    if i == 5:
                        AG[5,ii,jj]=F5[ii,jj]
                    if i == 6:
                        AG[6,ii,jj]=F6[ii,jj]
                    if i == 7:
                        AG[7,ii,jj]=F7[ii,jj]

        for ii in range(M):
            for jj in range(M):
                LI[ii,jj]=0.

        for i in range(M1):
            for ii in range(M):
                for jj in range(M):
                    LI[ii,jj]=LI[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))
        
        F3 = DMCRCR(LI,LI)
        LM=MMT
        
        for n, _ in enumerate(range(2), start=1):

            if n == 1:
                SA=-1.
            if n == 2:
                SA=1.

            for ii in range(M):
                for jj in range(M):
                    LU1[ii,jj]=SA*MMT*LU[ii,jj]


            EVU = DEVLCG(LU1)
            for i in range(M):
                for j in range(M):
                    F[i,j]=EVU[i]**j # (j-1) here was an expression (j-1)


            for ii in range(M):
                for jj in range(M):
                    F1[ii,jj]=F[ii,jj]


            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]
            
            A2 = DLFTCG(A1)
            DET10 = DLFDCG(A2); DET20 = 1 # DET10 and DET20 were the arguments of this function
            
            for j in range(M):
                for ii in range(M):
                    for jj in range(M):
                        F[ii,jj]=F1[ii,jj]


                for i in range(M):
#                     F = F.astype("complex128")
                    F[i,j]=EX1**EVU[i]


                for ii in range(M1):
                    for jj in range(M1):
                        A1[ii,jj]=F[ii,jj]


                A2 = DLFTCG(A1)
#                 A2 = A2.astype('float64')
                DET1[j] = DLFDCG(A2); DET2[j] = 1
            
            F2 = DMCRCR(LU1,LU1)
            F3 = DMCRCR(F2,LU1)
            F4 = DMCRCR(F3,LU1)
            F5 = DMCRCR(F4,LU1)
            F6 = DMCRCR(F5,LU1)
            F7 = DMCRCR(F6,LU1)

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):

                        if i == 0:
                            AG[0,ii,jj]=E[ii,jj]
                        if i == 1:
                            AG[1,ii,jj]=LU1[ii,jj]
                        if i == 2:
                            AG[2,ii,jj]=F2[ii,jj]
                        if i == 3:
                            AG[3,ii,jj]=F3[ii,jj]
                        if i == 4:
                            AG[4,ii,jj]=F4[ii,jj]
                        if i == 5:
                            AG[5,ii,jj]=F5[ii,jj]
                        if i == 6:
                            AG[6,ii,jj]=F6[ii,jj]
                        if i == 7:
                            AG[7,ii,jj]=F7[ii,jj]

            for ii in range(M):
                for jj in range(M):
                    LU2[ii,jj]=0.

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        LU2[ii,jj]=LU2[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))

            if n == 2:
                F2 = DMCRCR(LU3,LU2)
                continue
            else:
                for ii in range(M):
                    for jj in range(M):
                        LU3[ii,jj]=LU2[ii,jj]

        for n in range(2):
            if n == 0:
                SA=-1
            if n == 1:
                SA=1

            for ii in range(M):
                for jj in range(M):
                    LI1[ii,jj]=SA*MMT*LI[ii,jj]

            EVI = DEVLCG(LI1)

            for i in range(M):
                for j in range(M):
                    F[i,j]=EVI[i]**j # J-1 in original

            for ii in range(M):
                for jj in range(M):
                    F1[ii,jj]=F[ii,jj]

            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET10 = DLFDCG(A2); DET20 = 1
            
            for j in range(M):
                for ii in range(M):
                    for jj in range(M):
                        F[ii,jj]=F1[ii,jj]

                for i in range(M):
                    F[i,j]=EX1**EVI[i]

                for ii in range(M1):
                    for jj in range(M1):
                        A1[ii,jj]=F[ii,jj]

                A2 = DLFTCG(A1)
                DET1[j] = DLFDCG(A2); DET2[j] = 1


            F2 = DMCRCR(LI1,LI1)
            F3 = DMCRCR(F2,LI1)
            F4 = DMCRCR(F3,LI1)
            F5 = DMCRCR(F4,LI1)
            F6 = DMCRCR(F5,LI1)
            F7 = DMCRCR(F6,LI1)
            
            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        if i == 0:
                            AG[0,ii,jj]=E[ii,jj]
                        if i == 1:
                            AG[1,ii,jj]=LI1[ii,jj]
                        if i == 2:
                            AG[2,ii,jj]=F2[ii,jj]
                        if i == 3:
                            AG[3,ii,jj]=F3[ii,jj]
                        if i == 4:
                            AG[4,ii,jj]=F4[ii,jj]
                        if i == 5:
                            AG[5,ii,jj]=F5[ii,jj]
                        if i == 6:
                            AG[6,ii,jj]=F6[ii,jj]
                        if i == 7:
                            AG[7,ii,jj]=F7[ii,jj]

            for ii in range(M):
                for jj in range(M):
                    LI2[ii,jj]=0

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        LI2[ii,jj]=LI2[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))
            
            if n == 2:
                continue
            else:
                for ii in range(M):
                    for jj in range(M):
                        LI3[ii,jj]=LI2[ii,jj]


        F2 = DMCRCR(LI3,LI2)
        
######################## !!!!!!! PROBLEMS START HERE WITH GG, GG1 and GG2 !!!!!!!!! #################
        
        for i in range(M20):
            for j in range(M20):
                GG[i,j]=0.
                GG1[i,j]=0.
                GG2[i,j]=0.

        for i in range(M):
            GG[i,i]=1.
            GG[i,i+M]=1.
            GG[i+M,i+2*M]=1.
            GG[i+M,i+3*M]=1.
            GG1[i,i]=1.
            GG1[i,i+M]=1.
            GG1[i+M,i+2*M]=1.
            GG1[i+M,i+3*M]=1.
        
        for i in range(M):
            for j in range(M):
                GG[i+2*M,j]=-LU[i,j]
                GG[i+2*M,j+M]=LU[i,j]
                GG[i+3*M,j+2*M]=-1*LI[i,j]
                GG[i+3*M,j+3*M]=+LI[i,j]

                GG1[i+2*M,j]=LU3[i,j]
                GG1[i+2*M,j+M]=LU2[i,j]
                GG1[i+3*M,j+2*M]=LI3[i,j]
                GG1[i+3*M,j+3*M]=LI2[i,j]

        GG2 = DLINCG(M20,GG1) # Look for M20

        
#         print("GG :", GG)
#         print("GG1 :", GG1)
#         print("GG2 :", GG2)

    
        for i in range(M):
            for j in range(M):
                HH11[i,j]=GG2[i,j]
                HH12[i,j]=GG2[i,j+M]
                HH13[i,j]=GG2[i,j+2*M]
                HH14[i,j]=GG2[i,j+3*M]
                HH21[i,j]=GG2[i+M,j]
                HH22[i,j]=GG2[i+M,j+M]
                HH23[i,j]=GG2[i+M,j+2*M]
                HH24[i,j]=GG2[i+M,j+3*M]
                HH31[i,j]=GG2[i+2*M,j]
                HH32[i,j]=GG2[i+2*M,j+M]
                HH33[i,j]=GG2[i+2*M,j+2*M]
                HH34[i,j]=GG2[i+2*M,j+3*M]
                HH41[i,j]=GG2[i+3*M,j]
                HH42[i,j]=GG2[i+3*M,j+M]
                HH43[i,j]=GG2[i+3*M,j+2*M]
                HH44[i,j]=GG2[i+3*M,j+3*M]
        
        
#         print("HH11 :", HH11);
#         print("HH12 :", HH12);
#         print("HH13 :", HH13);
#         print("HH14 :", HH14);
#         print("HH21 :", HH21);
#         print("HH22 :", HH22);
#         print("HH23 :", HH23);
#         print("HH24 :", HH24);
#         print("HH31 :", HH31);
#         print("HH32 :", HH32);
#         print("HH33 :", HH33);
#         print("HH34 :", HH34);
#         print("HH41 :", HH41);
#         print("HH42 :", HH42);
#         print("HH43 :", HH43);
#         print("HH44 :", HH44);

        
        
        F = DMCRCR(LU3,HH11)
        HH11 = DMCRCR(LU,F)
        F = DMCRCR(LU2,HH21)
        HH21 = DMCRCR(LU,F)
        F = DMCRCR(LU3,HH13)
        HH13 = DMCRCR(LU,F)
        F = DMCRCR(LU2,HH23)
        HH23 = DMCRCR(LU,F)
        F = DMCRCR(LI3,HH32)
        HH32 = DMCRCR(LI,F)
        F = DMCRCR(LI2,HH42)
        HH42 = DMCRCR(LI,F)
        F = DMCRCR(LI3,HH34)
        HH34 = DMCRCR(LI,F)
        F = DMCRCR(LI2,HH44)
        HH44 = DMCRCR(LI,F)
        
#         print("HH11: ", HH11)
#         print("HH21: ", HH21)
#         print("HH13: ", HH13)
#         print("HH23: ", HH23)
#         print("HH32: ", HH32)
#         print("HH42: ", HH42)
#         print("HH34: ", HH34)
#         print("HH44: ", HH44)
#         print("F: ", F)
        
#         for i in range(M10):
#             for j in range(M20):
#                 GG3[i,j]=0.0

        for i in range(M):
            for j in range(M):
                GG3[i,j]=-1*HH11[i,j]+HH21[i,j]
                GG3[i,j+2*M]=-1*HH13[i,j]+HH23[i,j]
                GG3[i,j+3*M]=-1*Z[i,j]
                GG3[i+M,j+M]=-1*HH32[i,j]+HH42[i,j]
                GG3[i+M,j+2*M]=-1*Y[i,j]
                GG3[i+M,j+3*M]=-1*HH34[i,j]+HH44[i,j]
        
        K1=0
        K0=0

        for j in range(M20):

            if IH[j] == 1:
                K1=K1+1
            if IH[j] == 0:
                if IH[j] == 0:
                    K0=K0+1
                if IH[j] == 1:
                    continue
                   
                for i in range(M10):
                    GG5[i,K0-1]=GG3[i,j]

            else:
                for i in range(M10):
                    GG4[i,K1-1]=-1*GG3[i,j] # Original was -GG3[i,j]
                    B10[K1-1]=B5[j]

                    
        #  !!!!!!!!!!!!!!!!!!!!        B10          !!!!!!!!!!!!!!!!!!!!!!!!


#         print("B10: ", B10)
#         print("GG4: ", GG4)
        B6 = DMUCRV(GG4,B10)     
#         print("B6: ", B6)
#         print("GG5: ", GG5)
        B7 = DLSLCG(GG5,B6)
        
#         print("B7: ", B7)  # B7 is the REAL PROBLEM. HAS TO BE CORRECTED   !!!!!!!!!!!!!!!!!!
        
#         print("GG5: ", GG5)
#         print("GG4: ", GG4)
#         print("B10: ", B10)
#         print("B6: ", B6)
#         print("B7: ", B7)
        
        K1=0

        for j in range(M20):
            if IH[j] == 0:
                K1=K1+1
            if IH[j] == 1:
                continue

            B5[j]=B7[K1-1]
#         print("B5: ", B5)  # B5 is the REAL PROBLEM. HAS TO BE CORRECTED   !!!!!!!!!!!!!!!!!!
        for i in range(M):
            UK1[i]=B5[i]
            AIK1[i]=B5[i+M]
        
#         print("Z: ", Z)
#         print("Y: ", Y)
#         print("AIK1: ", AIK1)   # AIK1 SHOLD BE CORRECTED !!!!!!!!!!!!!!!!!!!!!!!!!!
#         print("UK1: ", UK1)     # UK1 SHOLD BE CORRECTED !!!!!!!!!!!!!!!!!!!!!!!!!!
        
        AA = DMUCRV(Z,AIK1)
        BB = DMUCRV(Y,UK1)
        CC = DMCRCR(LI,LI3)
        DD = DMCRCR(LI,LI2)
        
#         print("AA:", AA)
#         print("BB:", BB)
#         print("CC:", CC)
#         print("DD:", DD)
        
        for i in range(M):
            B1[i]=UK1[i]
            B1[i+M]=AIK1[i]
            B1[i+2*M]=AA[i]
            B1[i+3*M]=BB[i]
        
        B4 = DLSLCG(GG,B1)
        
        for i in range(M):
            AA[i]=0.
            B[i]=0.
            B[i]=B4[i]

        AA = DMUCRV(LU3,B)

        for i in range(M):
            BB[i]=0.
            B[i]=0.
            B[i]=B4[i+M]

        BB = DMUCRV(LU2,B)

        for i in range(M):
            UX[i]=AA[i]+BB[i]
            if LM == MMT:
                UK1[i]=UX[i]
            UXM[i]=np.sqrt(UX[i].real**2+UX[i].imag**2)

        for i in range(M):
            AA[i]=0.
            B[i]=0.
            B[i]=B4[i+2*M]
        
        AA = DMUCRV(LI3,B)   # NEED TO CORRECT AA !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        for i in range(M):
            BB[i]=0.
            B[i]=0.
            B[i]=B4[i+3*M]
        
        BB = DMUCRV(LI2,B)   # NEED TO CORRECT BB !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        
        for i in range(M):
                    
            AIX[i]=AA[i]+BB[i]
            if LM == MMT:
                AIK1[i]=AIX[i]
            AIXM[i]=np.sqrt(AIX[i].real**2+AIX[i].imag**2)
            
########################### PROBLEM LIES IN    !!!!  AIX  !!!!  ###################################### 

#             print("AIX: ", AIX)
#             print("AIXM: ", AIXM)
#             print("R11: ", R11) # TOTALLY FINE ! 
            
            if i == 0 and LL == 0 and PR == 2:
                PPP1[NN,LL]=PPP1[NN,LL]+(AIXM[0]**2)/(2*R11[0])
            if i == 0 and LL > 0:
                PPP1[NN,LL]=PPP1[NN,LL]+(AIXM[0]**2)/(2*R11[0])
            if i == 1 and LL == 0 and PR == 2:
                PPP2[NN,LL]=PPP2[NN,LL]+(AIXM[1]**2)/(2*R11[1])
            if i == 1 and LL > 0:
                PPP2[NN,LL]=PPP2[NN,LL]+(AIXM[1]**2)/(2*R11[1])
            if i == 2 and LL == 0 and PR == 2:
                PPP3[NN,LL]=PPP3[NN,LL]+(AIXM[2]**2)/(2*R11[2])
            if i == 2 and LL > 0:
                PPP3[NN,LL]=PPP3[NN,LL]+(AIXM[2]**2)/(2*R11[2])
            if i == 3 and LL == 0 and PR == 2:
                PPP4[NN,LL]=PPP4[NN,LL]+(AIXM[3]**2)/(2*R11[3])
            if i == 3 and LL > 0:
                PPP4[NN,LL]=PPP4[NN,LL]+(AIXM[3]**2)/(2*R11[3])
            if i == 4 and LL == 0 and PR == 2:
                PPP5[NN,LL]=PPP5[NN,LL]+(AIXM[4]**2)/(2*R11[4])
            if i == 4 and LL > 0:
                PPP5[NN,LL]=PPP5[NN,LL]+(AIXM[4]**2)/(2*R11[4])
            if i == 5 and LL == 0 and PR == 2:
                PPP6[NN,LL]=PPP6[NN,LL]+(AIXM[5]**2)/(2*R11[5])
            if i == 5 and LL > 0:
                PPP6[NN,LL]=PPP6[NN,LL]+(AIXM[5]**2)/(2*R11[5])
            if i == 6 and LL == 0 and PR == 2:
                PPP7[NN,LL]=PPP7[NN,LL]+(AIXM[6]**2)/(2*R11[6])
            if i == 6 and LL > 0:
                PPP7[NN,LL]=PPP7[NN,LL]+(AIXM[6]**2)/(2*R11[6])
            if i == 7 and LL == 0 and PR == 2:
                PPP8[NN,LL]=PPP8[NN,LL]+(AIXM[7]**2)/(2*R11[7])
            if i == 7 and LL > 0:
                PPP8[NN,LL]=PPP8[NN,LL]+(AIXM[7]**2)/(2*R11[7])
            if LL == 0 and PR == 2:
                PPP[NN,LL]=PPP[NN,LL]+(AIXM[i]**2)/(2*R11[i])
            if LL > 0:
                PPP[NN,LL]=PPP[NN,LL]+(AIXM[i]**2)/(2*R11[i])
            if LL == 0 and PR == 1:
                PP1=PP1+(AIXM[i]**2)/(2*R11[i])
            if LL == 0 and PR == 2:
                PP2=PP2+(AIX[i]**2)/(2*R11[i])
            SM[i]=UX[i]*np.conj(AIX[i])/2
            
#             print(PPP1[NN, LL])

        # Writing UXM and AIXM values into 10th and 11th modules    

In [32]:
# !!!  470th loop stands for 1th or one prisoed of podstanst  !!!

# 11 - 13
for i in range(560+1):
    for p in range(3):
        UM[i,0,p] = funu[i,p]*np.sqrt(2) 
        FUM[i,0,p]=fu[i,p]*np.pi/180 
        UM1[i,0,p]=UM[i,0,p]*np.cos(FUM[i,0,p]) 
        UM2[i,0,p]=UM[i,0,p]*np.sin(FUM[i,0,p])     
        
# 13 - 14
for i in range(560+1):
    for p in range(3):
        for k in range(50-1):
            UM[i,k+1,p]=UM[i,k+1,p]*UM[i,0,p]/100.
            FUM[i,k+1,p]=FUM[i,k+1,p]*np.pi/180.
            UM1[i,k+1,p]=UM[i,k+1,p]*np.cos(FUM[i,k+1,p])
            UM2[i,k+1,p]=UM[i,k+1,p]*np.sin(FUM[i,k+1,p])

# 14 - 16
for i in range(560+1):
    for p in range(3):
        AIM[i,0,p]=funi[i,p]*np.sqrt(2)
        FIM[i,0,p]=fi[i,p]*np.pi/180
        AIM1[i,0,p]=AIM[i,0,p]*np.cos(FIM[i,0,p])
        AIM2[i,0,p]=AIM[i,0,p]*np.sin(FIM[i,0,p])
        
# 16 - 117
for i in range(560+1):
    for p in range(3):
        for k in range(50-1):
            AIM[i,k+1,p]=AIM[i,k+1,p]*AIM[i,0,p]/100
            FIM[i,k+1,p]=FIM[i,k+1,p]*np.pi/180
            AIM1[i,k+1,p]=AIM[i,k+1,p]*np.cos(FIM[i,k+1,p])
            AIM2[i,k+1,p]=AIM[i,k+1,p]*np.sin(FIM[i,k+1,p])

In [33]:
#  !!!!!!!!!!!!!!!!!!!!!!!!!! WATCH OUT WITH UK1 and AIK1 !!!!!!!!!!!!!!!!!!!!!

In [28]:
# 117 - 1700 - 1500

print("!!! Started !!!")

MM, MPR, MTR, DT, MT = 5, 3, 1, 2.5, 5 #         <---- INPUT FROM THE USER
M = MPR + MTR     # thus is 4
MMT = MM / MT    # is 1

if M <= 6:
    M1 = M
if M > 6:
    M1 = 6

M10 = 2 * M # is 8
M20 = 4 * M # is 16

n = LoopI(0, 49) # OUTER LOOP
k = LoopI(0, 560) # INNTER LOOP
PR = LoopI(0, 5)
counter = 1

start = time.time()

while True:

    if k.the_end:
        break
    
    K = k.get_current_index(); N = n.get_current_index();
    
#     if N > 49:
#         N = 49
        
    UK1[0] = complex(UM1[K,N,0], UM2[K,N,0])
    UK1[1] = complex(UM1[K,N,1], UM2[K,N,1])
    UK1[2] = complex(UM1[K,N,2], UM2[K,N,2])
    UK1[3] = complex(0, 0)
    AIK1[0] = complex(AIM1[K,N,0], AIM2[K,N,0])
    AIK1[1] = complex(AIM1[K,N,1], AIM2[K,N,1])
    AIK1[2] = complex(AIM1[K,N,2], AIM2[K,N,2])
    AIK1[3] = complex(0, 0)

    if not (n.get_current_index() > 0) or (k.get_current_index() == 0 and PR.get_current_index() ==2):
#         print(counter); counter+=1
        UK10=(UK1[0]+UK1[1]+UK1[2])/3
        UK11=(UK1[0]+UK1[1]*AL+UK1[2]*AL**2)/3
        UK12=(UK1[0]+UK1[1]*AL**2+UK1[2]*AL)/3
        SKU2=np.sqrt(UK12.real**2+UK12.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
        SKU0=np.sqrt(UK10.real**2+UK10.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
        UK1[0]=UK11
        UK1[1]=UK11*AL**2
        UK1[2]=UK11*AL
        AIK10=(AIK1[0]+AIK1[1]+AIK1[2])/3
        AIK11=(AIK1[0]+AIK1[1]*AL+AIK1[2]*AL**2)/3
        AIK12=(AIK1[0]+AIK1[1]*AL**2+AIK1[2]*AL)/3
        SKI2=np.sqrt(AIK12.real**2+AIK12.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
        SKI0=np.sqrt(AIK10.real**2+AIK10.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
        AIK1[1]=AIK11
        AIK1[2]=AIK11*AL**2
        AIK1[3]=AIK11*AL

#     print("OUNTER, INNER", k.get_current_index(), n.get_current_indexS())
#     print(counter); counter+=1
    RASCHET(K, N)


    if k.get_current_index() == 0 and PR.get_current_index() == 1:
        pass
    if k.get_current_index() == 0 and PR.get_current_index() == 2:
        pass
    if k.get_current_index() == 0 and PR.get_current_index() == 1:
        PR.step_up_index()
        continue
    if PR.get_current_index() == 2:
        PR.set_current_index(0)

    if n.the_end:
        k.step_up_index()
        n.set_current_index(0)
    else:
        n.step_up_index()

print("!!! Finished !!!")
end = time.time()
print("Time took(sec): ", end - start)

!!! Started !!!
0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
3 22
3 23
3 24
3 25
3 26
3 27
3 28
3 29
3 30
3 31
3 32
3 33
3 34
3 35
3 36
3 37
3 38
3 39
3 40
3 41
3 42
3 43
3 44
3 45
3 46
3 47
3 48
3 49
4 0
4 1
4 2
4 3
4 4
4 5


KeyboardInterrupt: 

In [ ]:
# 1500 - 1501

for n in range(560):
    PRP = 0
    for k in range(50): # 1060'th loop
        PRP=PRP+PPP[n,k]
    RPR=0
    for k in range(1, 50): # 1061'st loop
        RPR=RPR+PPP[n,k]
    
    SS1=(PPP[n,0]/PPR1[n]-1)*100
    SS2=(RPR/PPR1[n])*100
    SS0=PPP[n,0]-PPR1[n]
    
    # WRITING INTO "PPP" and "PPPx" files corresponding PPP and PPPx values
#     print(PPP1[n, k])
    
# 1501 - 1052
WD0=0.
for k in range(50):
    WD[0,k]=0
    for i in range(560):
        WD0=WD0+PPP[i,k]*DT/60000
        WD[0,k]=WD[0,k]+PPP[i,k]*DT/60000

# 1052 - 1057
WD1=WD0-WD[0,0]
for k in range(50):
    WD[1,k]=WD[0,k]/WD0*100
WD4=0
for k in range(13, 50):
    WD4=WD4+WD[1,k]
WD10=0
for i in range(560):
    WD10=WD10+PPR1[i]*DT/6000ii0
 
# 1057 -1054
for i in range(560):
    PD[0,i] = 0
    PD[1,i] = 0
    for k in range(50):
        PD[0,i]=PD[0,i]+PPP[i,k]
    PD[1,i]=PD[0,i]-PPP[i,0]
    PD[2,i]=PD[1,i]/PD[0,i]*100

# WRITING CALCULATION RESULTS INTO 8th UNIT (Результаты расчета)

In [ ]:
for item in PPP[:560,1]:
    print(item)

In [ ]:
int(WD0) # HAVE TO BE AROUND ~ 16.79

In [22]:
x = 0
while True:
    print("Ohh", x)
    time.sleep(1)
    if x == 5:
        continue
    print("HEy")
    x += 1

Ohh 0
HEy
Ohh 1
HEy
Ohh 2
HEy
Ohh 3
HEy
Ohh 4
HEy
Ohh 5
HEy
Ohh 6
HEy
Ohh 7


KeyboardInterrupt: 